# Exploring OpenAI V1 functionality

On 11.06.23 OpenAI released a number of new features, and along with it bumped their Python SDK to 1.0.0. This notebook shows off the new features and how to use them with LangChain.

In [ ]:
!pip install -U openai "langchain>=0.0.331rc1" langchain-experimental

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage

## [Vision](https://platform.openai.com/docs/guides/vision)

OpenAI released multi-modal models, which can take a sequence of text and images as input.

In [3]:
chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=256)
chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "What is this image showing"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://python.langchain.com/assets/images/langchain_stack-da369071b058555da3d491a695651f15.jpg",
                        "detail": "auto",
                    },
                },
            ]
        )
    ]
)

AIMessage(content='The image appears to be a diagram of a software architecture or framework related to language processing, named "LangChain." This architecture seems to be designed to handle various aspects of natural language understanding or processing tasks. Here are the components as labeled in the diagram:\n\n1. **LangSmith**: This component is related to testing, evaluation, monitoring, feedback, and annotation. It also has a debugging aspect to it.\n2. **LangServe**: This is a service that can chain as REST API, indicating it might serve language processing capabilities over a network.\n3. **Templates**: Reference applications are mentioned here, which might be used as starting points or examples for building new applications.\n4. **Chains, agents, agent executors**: This part of the system handles common application logic, potentially acting as the middleware or the operational logic of the framework.\n5. **Model I/O**: Input/output management for the model, including prompts

## [OpenAI assistants](https://platform.openai.com/docs/assistants/overview)

> The Assistants API allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling


You can interact with OpenAI Assistants using OpenAI tools or custom tools. When using exclusively OpenAI tools, you can just invoke the assistant directly and get final answers. When using custom tools, you can run the assistant and tool execution loop using the built-in AgentExecutor or easily write your own executor.

Below we show the different ways to interact with Assistants. As a simple example, let's build a math tutor that can write and run code.

### Using only OpenAI tools

In [4]:
from langchain_experimental.openai_assistant import OpenAIAssistantRunnable


In [5]:
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})
output

[ThreadMessage(id='msg_RGOsJ2RBYp79rILrZ0NsAX68', assistant_id='asst_9Xb1ZgefoAbp2V5ddRlDGisy', content=[MessageContentText(text=Text(annotations=[], value='\\( 10 - 4^{2.7} \\) is approximately -32.2243.'), type='text')], created_at=1699385426, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_E2PRoP04ryly4p5ds5ek2qxs', thread_id='thread_pu9CpsYIWWZtxemZxpbYfhm4')]

### As a LangChain agent with arbitrary tools

Now let's recreate this functionality using our own tools. For this example we'll use the [E2B sandbox runtime tool](https://e2b.dev/docs?ref=landing-page-get-started).

In [7]:
from langchain.tools import E2BDataAnalysisTool

tools = [E2BDataAnalysisTool(api_key="...")]

In [8]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True
)

#### Using AgentExecutor

In [9]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's 10 - 4 raised to the 2.7"})

{'content': "What's 10 - 4 raised to the 2.7",
 'output': 'The result of \\(10 - 4\\) raised to the power of \\(2.7\\) is approximately \\(126.19\\).'}

#### Custom execution

In [19]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True
)

In [22]:
from langchain.schema.agent import AgentFinish

def execute_agent(agent, tools, input):
    tool_map = {tool.name: tool for tool in tools}
    response = agent.invoke(input)
    while not isinstance(response, AgentFinish):
        tool_outputs = []
        for action in response:
            tool_output = tool_map[action.tool].invoke(action.tool_input)
            print(action.tool, action.tool_input, tool_output, end="\n\n")
            tool_outputs.append({"output": tool_output, "tool_call_id": action.tool_call_id})
        response = agent.invoke({"tool_outputs": tool_outputs, "run_id": action.run_id, "thread_id": action.thread_id})
        
    return response

In [23]:
response = execute_agent(agent, tools, {"content": "What's 10 - 4 raised to the 2.7"})
print(response.return_values["output"])

e2b_data_analysis {'python_code': 'print((10 - 4) ** 2.7)'} {"stdout": "126.18518711065899", "stderr": "", "artifacts": []}

\( 10 - 4 \) raised to the power of 2.7 is approximately 126.185.


In [24]:
next_response = execute_agent(agent, tools, {"content": "now add 17.241", "thread_id": response.thread_id})
print(next_response.return_values["output"])

e2b_data_analysis {'python_code': 'result = (10 - 4) ** 2.7 + 17.241\nprint(result)'} {"stdout": "143.426187110659", "stderr": "", "artifacts": []}

\( (10 - 4)^{2.7} + 17.241 \) is approximately 143.426.


## [JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode)

Constrain the model to only generate valid JSON. Note that you must include a system message with instructions to use JSON for this mode to work.

Only works with certain models. 

In [16]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(
    response_format={"type": "json_object"}
)

output = chat.invoke(
    [
        SystemMessage(
            content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
        ),
        HumanMessage(
            content="Google was founded in the USA, while Deepmind was founded in the UK"
        ),
    ]
)
print(output.content)

{
  "companies": [
    {
      "name": "Google",
      "origin": "USA"
    },
    {
      "name": "Deepmind",
      "origin": "UK"
    }
  ]
}


In [17]:
import json

json.loads(output.content)

{'companies': [{'name': 'Google', 'origin': 'USA'},
  {'name': 'Deepmind', 'origin': 'UK'}]}

## [System fingerprint](https://platform.openai.com/docs/guides/text-generation/reproducible-outputs)

OpenAI sometimes changes model configurations in a way that impacts outputs. Whenever this happens, the system_fingerprint associated with a generation will change.

In [18]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
output = chat.generate(
    [
        [
            SystemMessage(
                content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
            ),
            HumanMessage(
                content="Google was founded in the USA, while Deepmind was founded in the UK"
            ),
        ]
    ]
)
print(output.llm_output)

{'token_usage': {'completion_tokens': 43, 'prompt_tokens': 49, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_eeff13170a'}
